In [ ]:
import veloxchem as vlx
from openmmdynamics3 import OpenMMDynamics

In [ ]:
first_xyz = '1.xyz' # Name of first xyz-file in quotes ex '1.xyz'
second_xyz = '2.xyz' # Name of second xyz-file in quotes ex '2.xyz'
identical_molecules = True # Set to True if the two molecules are identical, False if they are different
first_charge = 1 
second_charge = 1
first_multiplicity = 2
second_multiplicity = 2
medium = 'water' # gas or water
size_of_md_box = 4 # Cubic MD simulation box in nm
simulation_time = 1 # simulation time in nanoseconds

In [ ]:
#Display the first molecule:
molecule1 = vlx.Molecule.read_xyz_file(first_xyz)
print('Structure of the molecule entered: ')
molecule1.show(atom_indices=True)

In [ ]:
#Dispaly the second molecule:
molecule2 = vlx.Molecule.read_xyz_file(second_xyz)
print('Structure of the molecule entered: ')
molecule2.show(atom_indices=True)

In [ ]:
#Generate force field and pdb for first structure
ff_gen = vlx.ForceFieldGenerator()
molecule1.set_multiplicity(first_multiplicity)
molecule1.set_charge(first_charge)
basis = vlx.MolecularBasis.read(molecule1, 'sto-3g')
scf_drv = vlx.ScfUnrestrictedDriver()
scf_drv.ostream.mute()
scf_drv.conv_thresh = 1e-3
scf_drv.max_iter = 500
scf_drv.xcfun = 'b3lyp'
results = scf_drv.compute(molecule1, basis)
ff_gen.ostream.mute()
ff_gen.create_topology(molecule1, basis , results)
ff_gen.write_openmm_files('mol1','MO1')

In [ ]:
#Generate force field and pdb for second structure
if identical_molecules == True:
    ff_gen2 = vlx.ForceFieldGenerator()
    ff_gen2.ostream.mute()
    ff_gen2.create_topology(molecule2, no_resp=True)
    ff_gen2.write_openmm_files('mol2','MO2')
else:
    ff_gen2 = vlx.ForceFieldGenerator()
    molecule2.set_multiplicity(second_multiplicity)
    molecule2.set_charge(second_charge)
    basis2 = vlx.MolecularBasis.read(molecule2, 'sto-3g')
    scf_drv2 = vlx.ScfUnrestrictedDriver()
    scf_drv2.ostream.mute()
    scf_drv2.conv_thresh = 1e-3
    scf_drv2.max_iter = 500
    scf_drv2.xcfun = 'b3lyp'
    results2 = scf_drv2.compute(molecule2, basis2)
    ff_gen2.ostream.mute()
    ff_gen2.create_topology(molecule2, basis2 , results2)
    ff_gen2.write_openmm_files('mol2','MO2')

In [ ]:
#Run molecular dynamics
opm_dyn = OpenMMDynamics()
opm_dyn.box_size = size_of_md_box
pdb_files = ['mol1.pdb', 'mol2.pdb']
xml_files = ['mol1.xml']
opm_dyn.build_custom_system(pdb_files, xml_files, phase=medium)
steps=simulation_time*500000
snap=200

# Vi kan nu köra en kort simulering för att se att allt fungerar!!!
opm_dyn.run_md(ensemble='NVT', 
               temperature=300,
               timestep=2.0,
               nsteps=steps,
               snapshots=snap)

In [ ]:
#Visualize dynamics
import nglview
import mdtraj as md

traj = md.load("trajectory.pdb", top="trajectory.pdb")
traj.image_molecules(anchor_molecules=[set(traj.topology.residue(0).atoms)])
view = nglview.show_mdtraj(traj)
view.clear_representations()
view.add_representation("ball+stick", selection="water", opacity=0.2)
view.add_spacefill("not water")
view.center()
view